## Preparando ambiente

In [2]:
import spacy
from spacy import displacy
from IPython.display import SVG, display

## Introdução ao Spacy

Diferente do curso original, aqui tentarei utilizar exemplos e linguagem em português para comparar a qualidade dos resultados.

In [2]:
nlp = spacy.blank('pt')

É possível interagir com os tokens e identificar o seu tipo (número, pontuação, etc).

In [16]:
doc = nlp('Olá, mundo! Um, dois e três')

for token in doc:
    print(f'{token.text} \t=> Is punct: {token.is_punct}\t| number: {token.like_num}\t| stop: {token.is_stop}')

Olá 	=> Is punct: False	| number: False	| stop: False
, 	=> Is punct: True	| number: False	| stop: False
mundo 	=> Is punct: False	| number: False	| stop: False
! 	=> Is punct: True	| number: False	| stop: False
Um 	=> Is punct: False	| number: True	| stop: True
, 	=> Is punct: True	| number: False	| stop: False
dois 	=> Is punct: False	| number: True	| stop: True
e 	=> Is punct: False	| number: False	| stop: True
três 	=> Is punct: False	| number: True	| stop: True


* Identifica tokens que se referem a números mesmo que esteja escrito por extenso
* Identifica stopwords

## Trained pipelines

In [19]:
nlp = spacy.load('pt_core_news_sm')

In [21]:
text = "Ela comeu a pizza"
doc = nlp(text)

for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)

Ela PRON nsubj comeu
comeu VERB ROOT comeu
a DET det pizza
pizza NOUN obj comeu


In [42]:
spacy.explain("nsubj")

'nominal subject'

Identifica as partes da fala (POS). É possível identificar o tipo de palavra (verbo, substantivo, etc) com um vocabulário em português previamente treinado. Estou usando aqui o modelo small, mas existem outros modelos maiores que podem ser utilizados, como o Large (mesmo nome com final `lg` ao invés de `sm`). A alteração do modelo pode apresentar algumas alterações nos resultados de POS. O significado de cada tag de POS pode ser encontrado [aqui](https://universaldependencies.org/u/pos/DET.html). Além disso, basta executar o comando `spacy.explain('ADJ')` para obter a explicação de cada tag.

Além disso, as propriedades `dep_` e `head` trazem as dependências entre as palavras.

In [41]:
nlp = spacy.load('pt_core_news_lg')
text = "Venha ver as promoções de celulares samsung na americanas nessa black friday cerveja. Apple está pensando em comprar empresa do Brasil por R$ 1 bilhão. Apple está querendo comprar uma startup do Reino Unido por 100 milhões de dólares."
doc = nlp(text)

for token in doc:
    print(f'{token.text}\t{token.pos_}\t{token.dep_}\t{token.head.text}')

print('\n----------------------\n')

for ent in doc.ents:
    print(ent.text, ent.label_)

Venha	VERB	ROOT	Venha
ver	VERB	xcomp	Venha
as	DET	det	promoções
promoções	NOUN	obj	ver
de	ADP	case	celulares
celulares	NOUN	nmod	promoções
samsung	PROPN	appos	promoções
na	ADP	case	americanas
americanas	NOUN	obl	ver
nessa	ADP	case	black
black	PROPN	obl	ver
friday	PROPN	flat:name	black
cerveja	ADJ	amod	black
.	PUNCT	punct	Venha
Apple	PROPN	nsubj	pensando
está	AUX	aux	pensando
pensando	VERB	ROOT	pensando
em	SCONJ	mark	comprar
comprar	VERB	xcomp	pensando
empresa	NOUN	obj	comprar
do	ADP	case	Brasil
Brasil	PROPN	nmod	empresa
por	ADP	case	R$
R$	SYM	obl	comprar
1	NUM	nummod	R$
bilhão	NUM	flat	1
.	PUNCT	punct	pensando
Apple	PROPN	nsubj	querendo
está	AUX	aux	querendo
querendo	VERB	ROOT	querendo
comprar	VERB	xcomp	querendo
uma	DET	det	startup
startup	NOUN	obj	comprar
do	ADP	case	Reino
Reino	PROPN	nmod	startup
Unido	PROPN	flat:name	Reino
por	ADP	case	100
100	NUM	obl	comprar
milhões	NUM	flat	100
de	ADP	case	dólares
dólares	NOUN	nmod	100
.	PUNCT	punct	querendo

----------------------

Venha LOC
App

Note acima que a detecção de entidades para português é bastante incompleta. Especialmente se comparada com a detecção em inglês abaixo.

In [40]:
nlp = spacy.load('en_core_web_sm')

text = "Apple is looking at buying U.K. startup for $1 billion and samsung is waiting to see what happens"
doc = nlp(text)
for token in doc:
    print(f'{token.text}\t{token.pos_}\t{token.dep_}\t{token.head.text}')

print('\n----------------------\n')

for ent in doc.ents:
    print(ent.text, ent.label_)

Apple	PROPN	nsubj	looking
is	AUX	aux	looking
looking	VERB	ROOT	looking
at	ADP	prep	looking
buying	VERB	pcomp	at
U.K.	PROPN	dobj	buying
startup	NOUN	dep	looking
for	ADP	prep	startup
$	SYM	quantmod	billion
1	NUM	compound	billion
billion	NUM	pobj	for
and	CCONJ	cc	startup
samsung	PROPN	nsubj	waiting
is	AUX	aux	waiting
waiting	VERB	conj	startup
to	PART	aux	see
see	VERB	xcomp	waiting
what	PRON	nsubj	happens
happens	VERB	ccomp	see

----------------------

Apple ORG
U.K. GPE
$1 billion MONEY
samsung ORG


Uma coisa a se estudar é a possibilidade de incluir entidades dentro da busca. Por exemplo, se eu quiser buscar por um verbo que esteja relacionado a uma entidade específica, como "comprar" e "casa", eu poderia buscar por `VERB` e `casa` e verificar se o verbo está relacionado à entidade `casa`. Além disso, [essa issue](https://github.com/explosion/spaCy/discussions/10232) parece indicar que é possível adicionar até mesmo padrões, como a identificação de um CPF, por exemplo.

### Visualizando as dependências

A visualização abaixo é para entender um pouco melhor como a biblioteca entende as relações entre as palavras. Está sendo utilizado o modelo `small`, mas é possível utilizar o `large` para obter resultados mais precisos.

In [48]:
nlp = spacy.load('pt_core_news_sm')
text = "Venha conhecer as nossas promoções de celulares samsung para este natal."
doc = nlp(text)
    
def showSVG(s):
  display(SVG(s))

graph01 = displacy.render(doc)
showSVG(graph01)

<IPython.core.display.SVG object>

## Rule-based matching

Isso pode ser muito interessante para melhorar a busca do taggeamento dependendo de como ela for ser aplicada.

In [4]:
nlp = spacy.load('pt_core_news_lg')
text = "Venha conhecer as nossas promoções de celulares samsung para este natal."
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

In [5]:
nlp = spacy.load('en_core_web_sm')
text = "Come and discover our Samsung cell phone promotions for this Christmas."
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

Samsung ORG
this Christmas DATE


Note acima que nenhuma entidade foi reconhecida mesmo com o vocabulário de português maior. Por outro lado veja como a mesma frase traduzida para inglês é reconhecida com o vocabulário `small` do inglês. O matching por regras ajudaria a melhorar a identificação, além de poder filtrar por palavras que têm mais de um significado.

In [8]:
from spacy.matcher import Matcher

nlp = spacy.load('pt_core_news_lg')
text = "Venha conhecer as nossas promoções de celulares samsung e moto g para este natal."

pattern_samsung = [{"TEXT": "samsung"}]
pattern_motog = [{"TEXT": "moto"}, {"TEXT": "g"}]

matcher = Matcher(nlp.vocab)
matcher.add("SAMSUNG", [pattern_samsung])
matcher.add("MOTOG", [pattern_motog])

doc = nlp(text)
matches = matcher(doc)

for match_id, start, end in matches:    
    matched_span = doc[start:end]
    print(matched_span.text)

samsung 
moto g 


Ainda assim, note que não foi identificado diretamente um novo objeto, mas apenas um padrão que utiliza da inteligência dos objetos, como no exemplo abaixo:

In [10]:
nlp = spacy.load('en_core_web_sm')
pattern = [
    {"LEMMA": "buy"},
    {"POS": "DET", "OP": "?"},  # optional: match 0 or 1 times
    {"POS": "NOUN"}
]

matcher = Matcher(nlp.vocab)
matcher.add("BUY", [pattern])

doc = nlp("I bought a smartphone. Now I'm buying apps.")

matches = matcher(doc)

for match_id, start, end in matches:    
    matched_span = doc[start:end]
    print(matched_span.text)

bought a smartphone
buying apps


Acima é possível perceber que o `matcher` identificou o padrão de que o verbo `comprar` está relacionado com o objeto `casa`. Isso pode ser muito útil para identificar padrões de busca. Abaixo podemos ver que é possível até mesmo fazer combinações semelhantes a RegEx mas de forma muito mais simples:

In [11]:
nlp = spacy.load('pt_core_news_lg')
pattern = [
    {"TEXT": "Windows"},
    {"IS_DIGIT": True}
]

matcher = Matcher(nlp.vocab)
matcher.add("WINDOWS", [pattern])

doc = nlp("As melhores versões do Windows foram: Windows 7, Windows 10 e Windows 11")

matches = matcher(doc)

for match_id, start, end in matches:    
    matched_span = doc[start:end]
    print(matched_span.text)

Windows 7
Windows 10
Windows 11


Note que apenas os Windows acompanhados de um dígito foram encontrados. A palavra sozinha não fez o match com a regra.

In [12]:
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "i downloaded Fortnite on my laptop and can't open the game at all. Help? "
    "so when I was downloading Minecraft, I got the Windows version where it "
    "is the '.zip' folder and I used the default program to unpack it... do "
    "I also need to download Winzip?"
)

pattern = [{"LEMMA": "download"}, {"POS": "PROPN"}]

matcher.add("DOWNLOAD_THINGS_PATTERN", [pattern])
matches = matcher(doc)

for match_id, start, end in matches:    
    matched_span = doc[start:end]
    print(matched_span.text)

downloaded Fortnite
downloading Minecraft
download Winzip


Acima, agora em inglês, apenas as palavras com a raíz "download" seguidas de nome próprio. Abaixo todos os nomes próprios identificados:

In [14]:
nlp = spacy.load('pt_core_news_lg')
matcher = Matcher(nlp.vocab)

doc = nlp(
    "aqui na americanas você tem as melhores promoções de celulares samsung e motorola, além das lavadores consul e electrolux"
)

pattern = [{"POS": "PROPN"}]

matcher.add("PROPN_PATTERN", [pattern])
matches = matcher(doc)

for match_id, start, end in matches:    
    matched_span = doc[start:end]
    print(matched_span.text)

samsung
motorola
electrolux


A identificação dos nomes próprios em português não é perfeita, mas poderia ser usada para identificar possíveis tags não encontradas pelo taggeamento cadastrado.

In [30]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

pattern = nlp("Golden Retriever")
matcher.add("DOG", [pattern])
doc = nlp("Eu tenho um Golden Retriever")

# Iterate over the matches
for match_id, start, end in matcher(doc):
    # Get the matched span
    span = doc[start:end]
    print("Matched span:", span.text)

Matched span: Golden Retriever


Acima vemos que é possível também fazer um Matcher para frases inteiras. O código abaixo de exemplo da documentação é um uso interessante:

```python	
import json
import spacy

with open("exercises/en/countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())

nlp = spacy.blank("en")
doc = nlp("Czech Republic may help Slovakia protect its airspace")

# Import the PhraseMatcher and initialize it
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

# Create pattern Doc objects and add them to the matcher
# This is the faster version of: [nlp(country) for country in COUNTRIES]
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", patterns)

# Call the matcher on the test document and print the result
matches = matcher(doc)
print([doc[start:end] for match_id, start, end in matches])
```

Uma lista de tokens pré definidos (nomes dos países) é informada e o match é feito diretamente por essa lista. Isso é basicamente o que fazemos hoje. É necessário apenas estudar se o desempenho do que o spacy faz é melhor.

## Adicionando novas entidades

In [21]:
from spacy.tokens import Span

nlp = spacy.load('pt_core_news_lg')

doc = nlp(
    "aqui na americanas você tem as melhores promoções de celulares samsung e motorola, além das lavadores consul e electrolux"
)

for ent in doc.ents:
    print(ent.text, ent.label_)

span = Span(doc, 10, 11, label="ORG")
doc.ents = list(doc.ents) + [span]

for ent in doc.ents:
    print(ent.text, ent.label_)


motorola ORG
samsung ORG
motorola ORG


Acima é adicionado via um documento. Como adicionar para o vocabulário inteiro? Vamos começar criando uma lista de marcas:

In [38]:
brands = [
    "samsung",
    "motorola",
    "fruki",
    "electrolux",
    "consul"
]

In [39]:
text = "Promoção lojas americanas com celulares samsung e motorola, além das lavadores consul e electrolux. Venha, estamos esperando você com uma fruki gelada!"

In [36]:
nlp = spacy.load('pt_core_news_lg')

doc = nlp(
    "aqui na americanas você tem as melhores promoções de celulares samsung e motorola, além das lavadores consul e electrolux"
)
for ent in doc.ents:
    print(ent.text, ent.label_)

motorola ORG


Acima vemos que apenas a entidade motorola foi identificada.

In [42]:
from spacy.language import Language
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

brand_patterns = list(nlp.pipe(brands))
matcher = PhraseMatcher(nlp.vocab)
matcher.add("BRANDS", brand_patterns)

@Language.component("brand_component")
def animal_component_function(doc):
    # Apply the matcher to the doc
    matches = matcher(doc)
    # Create a Span for each match and assign the label "ANIMAL"
    spans = [Span(doc, start, end, label="ORG") for match_id, start, end in matches]
    # Overwrite the doc.ents with the matched spans
    doc.ents = spans
    return doc

if "brand_component" not in nlp.pipe_names:
    nlp.add_pipe("brand_component", before="ner")
print(nlp.pipe_names)

# Process the text and print the text and label for the doc.ents
doc = nlp(
    "aqui na americanas você tem as melhores promoções de celulares samsung e motorola, além das lavadores consul e electrolux"
)

print([(ent.text, ent.label_) for ent in doc.ents])

['tok2vec', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'brand_component', 'ner']
[('samsung', 'ORG'), ('motorola', 'ORG'), ('consul', 'ORG'), ('electrolux', 'ORG')]


O problema aqui continua que as entidades estão sendo adicionadas não via vocabulário, mas dinamicamente na execução do pipeline de cada documento. Da forma que está já seria útil para o taggeamento hoje, mas não o suficiente.

## Similaridade

A similaridade nessa biblioteca é calculada usando Word2Vec em vocabulário já treinado. Utiliza a distância de cosseno.

In [27]:
doc1 = nlp("Eu comprei um smartphone da Samsung")
doc2 = nlp("Ela ganhou um celular da LG")
print(doc1.similarity(doc2))

0.6336954300310972


Abaixo é possível ver como obter o vetor de uma palavra.

In [28]:
comprei_vector = doc1[1].vector
print(comprei_vector)

[ 0.96909   -1.8741    -1.6277     1.4424    -0.81577   -1.8003
 -0.69867   -3.8572     2.1692     0.36282    0.8312    -0.72865
 -1.8473    -3.1222    -0.98182    0.048301   1.686     -1.0379
 -0.608     -0.65527    1.5713     1.8352     1.3512    -0.50214
 -2.403     -1.1599    -0.32929    0.81169   -1.381      2.3942
 -0.52012    2.195     -2.0804     0.77829   -3.1261     0.14672
 -1.567      2.106      0.5394    -2.3987     0.39329   -0.9931
  0.10342    0.41437    0.073363   1.4713    -1.7604    -0.75015
 -2.8503    -2.5687     0.85283   -0.14295   -0.66235    0.45273
  0.29417   -0.38988    2.6156    -1.0853    -1.6852    -0.05728
  0.67298   -1.293     -1.57       0.50339   -2.5344    -2.0501
 -0.52929   -0.34608   -0.67849    0.11726    2.0476     0.30721
 -2.714     -0.15327   -1.1189     0.65284    3.0089     1.8972
 -0.13324    0.59238    0.57586   -3.7116     1.226      1.3121
 -0.98548    1.0672     0.66809   -0.45836   -0.09541   -2.0687
  1.3946     0.33464    0.63427  

Da mesma forma é possível obter o vetor de uma frase.

In [29]:
doc.vector

array([ 0.82877076, -1.4959415 , -0.01407002,  0.50191945, -1.5201298 ,
       -2.2141569 , -1.9054449 , -1.3089108 , -0.9012207 , -0.609249  ,
        1.409032  , -0.6197735 ,  0.29524034, -2.2386827 , -1.0073334 ,
        0.10729835,  2.5295959 ,  0.20271178, -0.11834935,  1.7934837 ,
       -1.4205139 ,  1.2539212 ,  0.18040785,  1.5367905 , -0.71002257,
        1.6561524 ,  0.7574853 ,  0.20762756, -1.140725  , -0.0794815 ,
       -0.6337527 ,  0.599849  , -0.9800525 , -0.9954756 ,  0.08243648,
       -0.89043844, -0.9970745 , -0.18366098,  0.32128188,  0.57742554,
        0.23275337,  0.44352174, -0.5531765 , -0.8138048 ,  0.514523  ,
       -0.621946  , -1.7735662 ,  0.00781301,  1.1821475 , -0.6922964 ,
       -1.0185323 , -1.6059405 , -1.5012765 , -0.3584405 , -0.04261879,
        0.23985079,  1.0476487 , -1.5052786 ,  0.2574795 ,  1.338814  ,
        0.8240164 ,  0.75161153,  0.91135967,  1.9343933 , -0.19888262,
        1.2405589 , -0.8107554 ,  0.88141143, -0.64680547, -0.30

## Pipelines

In [32]:
nlp = spacy.load('pt_core_news_lg')
nlp.pipeline[-1]

('ner', <spacy.pipeline.ner.EntityRecognizer at 0x2abe45bda80>)